In [11]:
import joblib
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, StringTensorType
import pandas as pd

load the scikit model

In [7]:
clf = joblib.load('../model.pkl')

some auxiliary function to help converting inputs to ONNX format

In [26]:
def convert_sklearn_onnx(clf, x_sample):
    inputs = convert_dataframe_schema(x_sample)
    onnx_model = convert_sklearn(clf, 'model_pipeline', inputs, target_opset=11)

    with open("model.onnx", "wb") as f:
        f.write(onnx_model.SerializeToString())

    return onnx_model


def convert_dataframe_schema(df, drop=None):
    inputs = []
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([None, 1])
        elif v == 'float64':
            t = FloatTensorType([None, 1])
        else:
            t = StringTensorType([None, 1])
        inputs.append((k, t))
    return inputs


def onnx_input(x):
    inputs = {c: x[c].values for c in x.columns}
    for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))
    return inputs

In [ ]:
x = pd.read_csv('../data/adult_test.csv').drop(['income'], axis=1)
x['age']=x['age'].astype(int)
x['hours_per_week']=x['hours_per_week'].astype(int)
x['capital_gain']=x['capital_gain'].astype(int)
x['capital_loss']=x['capital_loss'].astype(int)

model conversion (needs an input example)

In [48]:
onnx_model = convert_sklearn_onnx(clf, x)

Cool model Viz tool that shows computational graph

In [49]:
from onnx.tools.net_drawer import GetPydotGraph, GetOpNodeProducer
pydot_graph = GetPydotGraph(onnx_model.graph, name=onnx_model.graph.name, rankdir="TP",
                            node_producer=GetOpNodeProducer("docstring"))
pydot_graph.write_dot("graph.dot")

In [50]:
import os
os.system('dot -O -Tpng graph.dot')

0

inference time (onnxruntime is quite lightweight)

In [56]:
import onnxruntime as rt

In [57]:
sess = rt.InferenceSession('./model.onnx')


onnx_outputs = sess.run(None, onnx_input(x))


In [58]:
onnx_outputs

[array(['<=50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '>50K'],
       dtype=object),
 [{'<=50K': 0.5514597296714783, '>50K': 0.44854027032852173},
  {'<=50K': 0.6158027648925781, '>50K': 0.3841972351074219},
  {'<=50K': 0.7316201329231262, '>50K': 0.2683798670768738},
  {'<=50K': 0.6950892210006714, '>50K': 0.3049107491970062},
  {'<=50K': 0.6721135377883911, '>50K': 0.3278864324092865},
  {'<=50K': 0.5856322646141052, '>50K': 0.4143677353858948},
  {'<=50K': 0.7952150702476501, '>50K': 0.20478494465351105},
  {'<=50K': 0.7492754459381104, '>50K': 0.25072458386421204},
  {'<=50K': 0.965343713760376, '>50K': 0.034656260162591934},
  {'<=50K': 0.9376500248908997, '>50K': 0.06234995648264885},
  {'<=50K': 0.875234842300415, '>50K': 0.12476513534784317},
  {'<=50K': 0.9459554553031921, '>50K': 0.05404455587267876},
  {'<=50K': 0.8225024342536926, '>50K': 0.17749758064746857},
  {'<=50K': 0.5214500427246094, '>50K': 0.4785499572753906},
  {'<=50K': 0.9077594876289368, '>50K': 0.09224051